In [74]:
import cobra
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd 
import json
import re
import scipy.stats as st

In [5]:
from ECMpy_function import *

In [6]:
cobra.__version__

'0.26.2'

# Create sMOMENT model from ECMpy style ec-model

In [43]:
def convert_ECMpy_to_sMOMENT(json_model_file, output_filename):
    """using enzyme concentration constraint
    json model to create a COBRApy model in sMOMENT style.
    Modified from get_enzyme_constraint_model in ECMpy

    Arguments
    ----------
    * json_model_file: json Model file.
    * output_filename: Path to sMOMENT sbml file

    :return: Enzyme-constrained model in sMOMENT style.
    """

    dictionary_model = json_load(json_model_file)
    model = cobra.io.json.load_json_model(json_model_file)

    prot_pool = cobra.Metabolite(id = "prot_pool", name = "Pseudo-metabolite representing the preotein pool", 
                                 compartment = "c", charge = 0)
    for rxn in model.reactions:
        for eachr in dictionary_model['reactions']:
            if rxn.id == eachr['id']:
                if eachr['kcat_MW']:
                    coefficient = -1/float(eachr['kcat_MW'])
                    rxn.add_metabolites({prot_pool: coefficient})
                break
    # Add pool reaction
    lowerbound = dictionary_model['enzyme_constraint']['lowerbound']
    upperbound = dictionary_model['enzyme_constraint']['upperbound']
    prot_exchange = cobra.Reaction(id = "prot_pool_reaction", name = "prot_pool reaction for unmeasured proteins",
                                   lower_bound  = lowerbound, upper_bound=upperbound)
    prot_exchange.add_metabolites({prot_pool:1})
    model.add_reactions([prot_exchange])
    
    model.solver.update()
    model.id = "ecCGL1_sMOMENT_style"
    cobra.io.write_sbml_model(model, output_filename)
    return model

In [44]:
ecmodel_fn = 'models/c_glutamicum/smoment_iCW773.xml'
ecmodel = convert_ECMpy_to_sMOMENT('models/c_glutamicum/ecCGL1/iCW773_irr_enz_constraint_adj_PDH.json', ecmodel_fn)

In [ ]:
solution = ecmodel.optimize()
ecmodel.summary()

In [39]:
ecmodel.reactions.EX_glc_e_reverse.bounds = (0, 5.05)

In [ ]:
ecmodel.optimize()
print(ecmodel.summary())

## Load and run model in ECMpy to verify that results are identical

In [ ]:
ecModel_final_output_file = 'models/c_glutamicum/ecCGL1/iCW773_irr_enz_constraint_adj_PDH.json'
obj='CG_biomass_cgl_ATCC13032'
use_substrate='EX_glc_e'
enz_model=get_enzyme_constraint_model(ecModel_final_output_file)

enz_model.objective=obj
#Using only the provided substrate
[ori_obj_id,ori_substrate_id_list,ori_sub_concentration,ori_ATPM]=get_model_substrate_obj(enz_model)
for eachsubid in ori_substrate_id_list:
    if re.search('_reverse',eachsubid):
        r_id_new=eachsubid.split('_reverse')[0]
        enz_model.reactions.get_by_id(eachsubid).bounds = (0, 0) 
        enz_model.reactions.get_by_id(r_id_new).bounds = (0, 0)  
    else:
        r_id_new=eachsubid+'_reverse'
        enz_model.reactions.get_by_id(eachsubid).bounds = (0, 0) 
        enz_model.reactions.get_by_id(r_id_new).bounds = (0, 0) 
        
enz_model.reactions.get_by_id(use_substrate).bounds = (-5.05, 0)
enz_model.reactions.get_by_id(use_substrate+'_reverse').bounds = (0, 0)

enz_model_pfba_solution = cobra.flux_analysis.pfba(enz_model)
enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,enz_model_pfba_solution,fluxes_final_outfile,ecModel_final_output_file)
print(enz_model_pfba_solution.fluxes[obj])

In [ ]:
enz_model.optimize()
enz_model.summary()

# Analyses

In [45]:
# Load sMOMENT model
model = cobra.io.read_sbml_model('models/c_glutamicum/smoment_iCW773.xml')

In [ ]:
for r in model.exchanges:
    print(r, r.bounds)